In [1]:
def cv2_imshow(img):
    cv2.imshow('', img)
    cv2.waitKey()
    cv2.destroyAllWindows()
    
def imshow_plt(img):
    plt.figure(figsize=(15,15))
    if len(img.shape)==2:
        img = np.expand_dims(img, axis=-1)
    plt.imshow(img[:,:,0], cmap='gray')
    plt.show()

In [8]:
# import cv2
import numpy as np
import glob
import pandas as pd
import pydicom
from angle import *
import matplotlib.pyplot as plt


files = glob.glob('./4_result/exp1/result/AP/exp1_postprocessing/*.png')
img_ori_li = glob.glob('./3_deepdata/1_exp1/test/AP_pre_img/*.png')
names = np.load('./3_deepdata/1_exp1/npy/AP/test_name.npy')
angles = []

for i in range(len(files)):
    if i==1:
        break
    
#     i=113
    thickness = 1
    font = cv2.FONT_HERSHEY_SCRIPT_SIMPLEX  # hand-writing style font
    fontScale = 0.5
    print('*'*30)
    print(names[i])
#     dcm = pydicom.read_file('./5_angle/dcm/{}.dcm'.format(names[i]))
#     pixel_spacing = float(dcm[(0x0018, 0x1164)].value[0])
#     column = int(dcm[(0x0028, 0x0011)].value)
#     center_length = int((50/pixel_spacing)*(512/column))
    
    img = cv2.imread('./4_result/exp1/result/AP/exp1_postprocessing/{}_post.png'.format(names[i]))
    contour = max_contour(img)
    
    x_list, y_list, c_point, center_x, center_x2 = find_center_point(contour)
    c_angle = getAngle((center_x[0], center_x2[1]), center_x, center_x2)
#     print(c_angle)
    c_angle = 360-c_angle if c_angle>180 else c_angle
    if center_x[0]<center_x2[0]:
        if c_angle>0:
            c_angle = -c_angle
    
    step00 = img.copy()
    for c in range(len(c_point)-1):
        cv2.line(step00, c_point[c], c_point[c+1], (0,128,255),1, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step00.png', step00)
    
    step1 = img.copy()
    cv2.line(step1, center_x, center_x, (0,255,0),2, cv2.LINE_AA)
    cv2.line(step1, center_x2, center_x2, (0,255,0), 2, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step0.png', step1)
    cv2.line(step1, center_x, center_x2, (0,0,255), 1, cv2.LINE_AA)
    cv2.line(step1, (center_x[0], center_x[1]-20), (center_x[0], center_x2[1]+20), (255,0,0), 1, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step1.png', step1)
    
    img_ori = cv2.imread(img_ori_li[i])
    img_ori = cv2.resize(img_ori, (512,512))
    img_ori = rotate(img_ori, c_angle)
    
    img = rotate(img, c_angle)
    contour = max_contour(img)
    x_list, y_list, center_x, center_x2 = find_rotate_center_point(contour)
    
    step2 = img.copy()
    cv2.line(step2, center_x, center_x2, (0,255,200), 1, cv2.LINE_AA)
    cv2.line(step2, center_x, center_x, (0,255,0),2, cv2.LINE_AA)
    cv2.line(step2, center_x2, center_x2, (0,255,0), 2, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step2.png', step2)
#     print('c1_rotate : {}, c2 : {}'.format(center_x, center_x2))
    
    point_1 = find_ymax_point(contour)
    print('p1 : ', point_1) 
    
    step3 = img.copy()
    cv2.line(step3, point_1, point_1, (0,0,255),3, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step3.png', step3)
    
    LR=left_right(point_1,center_x)
    point_2, std_point = find_xpoint2_v2(img,contour,point_1,LR)
    print('p2 : ', point_2)
    
    step4 = img.copy()
    cv2.line(step4, std_point, std_point, (0,255,255), 2, cv2.LINE_AA)
    cv2.line(step4, point_2, point_2, (255,0,0), 3, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step4.png', step4)
    
    step4_1 = img.copy()
    cv2.line(step4_1, point_1, point_1, (0,0,255),3, cv2.LINE_AA)
    cv2.line(step4_1, point_2, point_2, (255,0,0), 3, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step4_1.png', step4_1)
    
    detect_range = 120
    left_increment_cnt = 0
    for m in range(center_x[0], center_x[0]-detect_range, -5):
        if m==center_x[0]:
            pre_value = img_ori[center_x[1]][m][0]
        else:
            if pre_value<img_ori[center_x[1]][m][0] and img_ori[center_x[1]][m][0]>80:
                left_increment_cnt+=1
            
            pre_value = img_ori[center_x[1]][m][0]
    print('left_increment_cnt: ', left_increment_cnt)

    right_increment_cnt = 0
    for m in range(center_x[0], center_x[0]+detect_range, 5):
        if m==center_x[0]:
            pre_value = img_ori[center_x[1]][m][0]
        else:
            if pre_value<img_ori[center_x[1]][m][0] and img_ori[center_x[1]][m][0]>80:
                right_increment_cnt+=1
            
            pre_value = img_ori[center_x[1]][m][0]
    print('right_increment_cnt: ', right_increment_cnt)
        
    if left_increment_cnt>right_increment_cnt:
        hands_cv = 1
    else:
        hands_cv = 0
    
    step5 = img_ori.copy()
    cv2.line(step5, center_x, (center_x[0]+detect_range, center_x[1]), (0,0,255),1, cv2.LINE_AA)
    cv2.line(step5, (center_x[0]-detect_range, center_x[1]), center_x, (255,0,0),1, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step5.png', step5)
    
    if hands_cv==0:
        if point_1[0]<point_2[0]:
            distal_side_point =  point_1
            ulna_side_point = point_2
        else:
            distal_side_point =  point_2
            ulna_side_point = point_1
    else:
        if point_1[0]<point_2[0]:
            distal_side_point =  point_2
            ulna_side_point = point_1
        else:
            distal_side_point =  point_1
            ulna_side_point = point_2
    
    step6 = img.copy()
    cv2.putText(step6, 'distal side point', (50,30), 0, fontScale, (0, 0,255), 1)
    cv2.putText(step6, 'ulna side point', (50,50), 0, fontScale, (255, 0,0), 1)
    cv2.line(step6, distal_side_point, distal_side_point, (0,0,255), 3, cv2.LINE_AA)
    cv2.line(step6, ulna_side_point, ulna_side_point, (255,0,0), 3, cv2.LINE_AA)
    cv2.imwrite('./5_angle/process/step6.png', step6)
            
    D = (distal_side_point[0], ulna_side_point[1])
    angle = getAngle(distal_side_point, ulna_side_point, D)
    if angle>180:
        angle = 360-angle
    if distal_side_point[1]>ulna_side_point[1]:
        angle = -angle
    
    angle = round(angle, 2)
    angles.append((names[i],angle))
    print('angle : ', angle)
    
    location=(ulna_side_point[0]-30, ulna_side_point[1]-10)    
    image = cv2.putText(img_ori, str(angle), location, font, fontScale, (255, 255,0), thickness)
    image = cv2.putText(img_ori, 'distal side point', (50,30), 0, fontScale, (255, 0,0), 1)
    image = cv2.putText(img_ori, 'ulna side point', (50,50), 0, fontScale, (0, 0,255), 1)
    
    image = cv2.line(img_ori, ulna_side_point, D, (200,255,0), 1)
    image = cv2.line(img_ori, ulna_side_point, distal_side_point, (255,200,0), 1)
    
    image = cv2.line(img_ori, distal_side_point, distal_side_point, (255,0,0), 2, cv2.LINE_AA)
    image = cv2.line(img_ori, ulna_side_point, ulna_side_point, (0,0,255), 2, cv2.LINE_AA)

    image = cv2.line(img_ori, center_x, center_x2, (200,255,0), 1, cv2.LINE_AA)
    image = cv2.line(img_ori, center_x, center_x, (0,255,0),2, cv2.LINE_AA)
    image = cv2.line(img_ori, center_x2, center_x2, (0,255,0), 2, cv2.LINE_AA)
    
#     image = cv2.line(img_ori, std_point, std_point, (0,255,0), 2, cv2.LINE_AA)
    image = cv2.line(img_ori, D, D, (0,255,0), 2, cv2.LINE_AA)

    image = array_to_img(image)
    image.save('./5_angle/process/step7.png')
    image.save("./5_angle/imgs_angle_AP_v2/angle_{}.png".format(names[i]))

******************************
1578645_X1A
p1 :  (223, 271)
p2 :  (266, 282)
left_increment_cnt:  1
right_increment_cnt:  6
angle :  14.35


In [37]:
import pandas as pd

df2 = pd.read_excel('./5_angle/GT.xlsx')
gt_li = df2['AP_GT'].tolist()

tmp =[]
for i in gt_li:
    
    if i>90:
        j=i-90
    elif i<90:
        j=90-i
#     print(i, round(j,2))
    tmp.append(round(j,2))
    
df = pd.DataFrame(angles, columns=['names', 'angle'])
# df2 = pd.read_excel('./test_ro.xlsx')
df['gt']=tmp
# df['ai_p']=tmp2
df.to_excel('./5_angle/imgs_angle_AP_v2.xlsx')
